In [ ]:
%matplotlib inline

In [ ]:
import os
import pickle
from math import floor
from random import shuffle
from statistics import variance
import math

from itertools import chain
from itertools import product
from itertools import groupby
from operator import itemgetter

import pandas as pd
import numpy as np

from geopy.distance import vincenty
import matplotlib.pyplot as plt

from utils import angle_diff_max

In [ ]:
with open('data_routes_pickle/routes_coord_cur', 'rb') as f:
    routes_coord_cur = pickle.load(f)

with open('data_routes_pickle/sim_matrix_sim_segments_1_mod23_cumul', 'rb') as f:
    sim_matrix = np.array(pickle.load(f))

with open('data_routes_pickle/dist_matrix', 'rb') as f:
    dist_matrix = np.array(pickle.load(f))

with open('data_routes_pickle/cluster_count_cur', 'rb') as f:
    cluster_count_cur = pickle.load(f)

with open('data_routes_pickle/idxs', 'rb') as f:
    idxs = pickle.load(f)

### Route angle avg/max on given dataset

In [ ]:
cluster_angles_avg = [0] * cluster_count_cur
cluster_lengths = [0] * cluster_count_cur
for route_idx in range(len(idxs)):
    cluster_idx = idxs[route_idx]
    route = routes_coord_cur[route_idx]
    
    cluster_angles_avg[cluster_idx] += angle_diff_max(route[:3])
    cluster_lengths[cluster_idx] += 1

for i in range(len(cluster_angles_avg)):
    cluster_angles_avg[i] /= cluster_lengths[i]

cluster_angles_avg

In [ ]:
cluster_angles_max = [0] * cluster_count_cur
cluster_max_route_idxs = [-1] * cluster_count_cur
for route_idx in range(len(idxs)):
    cluster_idx = idxs[route_idx]
    route = routes_coord_cur[route_idx]

    angle_diff_max_cur = angle_diff_max(route[:3])
    if angle_diff_max_cur > cluster_angles_max[cluster_idx]:
        cluster_angles_max[cluster_idx] = angle_diff_max_cur
        cluster_max_route_idxs[cluster_idx] = route_idx

cluster_angles_max, cluster_max_route_idxs

### Route length hist

In [ ]:
lengths = [len(route) for route in routes_coord_cur]
max_len = max(lengths)

hist_dict = {}
for idx in lengths:
    if idx not in hist_dict:
        hist_dict[idx] = 1
    else:
        hist_dict[idx] += 1
print(hist_dict)

plt.bar(list(hist_dict.keys()), list(hist_dict.values()))
plt.show()

# lengths

### Mean similarity rates per cluster

In [ ]:
route_idxs_per_cluster = [[] for i in range(cluster_count_cur)]
for r_i, c_i in enumerate(idxs):
    route_idxs_per_cluster[c_i].append(r_i)

[len(routes) for routes in route_idxs_per_cluster]

In [ ]:
mean_sims = [0] * cluster_count_cur
cells_counted = [0] * cluster_count_cur # basically same as n^2/2 + n/2

for cluster_idx, routes_i in enumerate(route_idxs_per_cluster):
    for i, route_idx_i in enumerate(routes_i):
        for route_idx_j in routes_i[i:]:
            mean_sims[cluster_idx] += sim_matrix[route_idx_i][route_idx_j]
            cells_counted[cluster_idx] += 1

mean_sims = [sim / cells_counted[i] for i, sim in enumerate(mean_sims)]
print(*mean_sims, sep='\n')

plt.bar(list(range(len(mean_sims))), mean_sims)
plt.show()